## Entraînement du modèle

In [39]:
import pandas as pd

data = pd.read_csv('./data/data.csv', sep = ',')

In [40]:
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

In [42]:
y = data['diagnosis']
X = data.drop(['diagnosis', 'id', 'Unnamed: 32'], axis=1)

In [43]:
# Transformation de y en numérique pour utilisation de Shapash
y = [0 if x == 'B' else 1 for x in y]

In [44]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, train_size=0.75, random_state=1)

rf = RandomForestClassifier(n_estimators=50, min_samples_leaf=3)

rf.fit(Xtrain, ytrain)

RandomForestClassifier(min_samples_leaf=3, n_estimators=50)

In [45]:
from sklearn.metrics import f1_score

y_pred_train = rf.predict(Xtrain)
print(f1_score(ytrain, y_pred_train, average='weighted'))

0.9929623676259919


In [46]:
y_pred_test = rf.predict(Xtest)
print(f1_score(ytest, y_pred_test, average='weighted'))

0.9438522359870675


In [47]:
import joblib

with open('./models/rf.pkl', 'wb') as model:
    joblib.dump(rf, model)

## Explicabilité des résultats obtenus sur le jeu de test

In [48]:
import shap
from shapash.explainer.smart_explainer import SmartExplainer

In [49]:
with open('./models/rf.pkl', 'rb') as model:
    rf = joblib.load(model)

In [50]:
ypred=pd.DataFrame(rf.predict(Xtest), columns=['pred'], index=Xtest.index)

In [51]:
#xpl = SmartExplainer(label_dict=label_dict, features_dict=feature_dict)
xpl = SmartExplainer()

In [52]:
'''xpl.compile(
            x=Xtest,
            model=rf,
            preprocessing=categ_encoding,
            y_pred=ypred,
            postprocessing=postprocessing
           )
'''

xpl.compile(
            x=Xtest,
            model=rf,
            y_pred=ypred,
           )

Backend: Shap TreeExplainer


In [53]:
xpl.to_pandas(max_contrib=3).head()

,pred,feature_1,value_1,contribution_1,feature_2,value_2,contribution_2,feature_3,value_3,contribution_3
421,0,perimeter_worst,114.1,-0.093674,texture_worst,18.34,0.081538,texture_mean,13.98,0.073348
47,1,concave points_worst,0.2088,0.150359,concave points_mean,0.0734,0.085447,concavity_mean,0.1226,0.059337
292,0,area_worst,585.4,0.059516,perimeter_worst,88.81,0.055235,concave points_worst,0.1056,0.040286
186,1,area_worst,1410.0,0.150622,perimeter_worst,139.2,0.135049,radius_worst,21.31,0.070254
414,0,area_worst,931.4,-0.074519,concave points_worst,0.06575,0.066547,concave points_mean,0.02739,0.065648


## Transformation pour la production

In [54]:
predictor = xpl.to_smartpredictor()

In [55]:
predictor.save('./models/predictor.pkl')

## Shapash Monitor

In [56]:
app = xpl.run_app()

/home/matos/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:1549: FutureWarning:

Using short name for 'orient' is deprecated. Only the options: ('dict', list, 'series', 'split', 'records', 'index') will be used in a future version. Use one of the above to silence this warning.

INFO:root:Your Shapash application run on http://matos-aspire-e5-522:8050/
INFO:root:Use the method .kill() to down your app.


Dash is running on http://0.0.0.0:8050/



INFO:shapash.webapp.smart_app:Dash is running on http://0.0.0.0:8050/



 * Serving Flask app "shapash.webapp.smart_app" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://0.0.0.0:8050/ (Press CTRL+C to quit)
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:53] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /assets/material-icons.css?m=1639154703.01868 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /_dash-component-suites/dash_renderer/react@16.v1_8_3m1639154693.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /_dash-component-suites/dash_renderer/prop-types@15.v1_8_3m1639154693.7.2.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /_dash-component-suites/dash_table/bundle.v4_11_0m1639154693.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /_dash-component-suites/dash_renderer/react-dom@16.v1_8_3m1639154693.14.0.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Dec/2021 15:39:54] "GET /_dash-component-suites/dash_renderer/polyfill@7.v1_8_3m1639154693.8.7.min.js HTTP/1.1" 2